In [10]:
import os
import numpy as np
import glob

# Set your dataset root and output path
DATASET_ROOT = "Stanford3dDataset_v1.2"
OUTPUT_ROOT = "processed_data"

# Define label mapping
CLASSES = [
    "ceiling", "floor", "wall", "beam", "column", "window",
    "door", "table", "chair", "sofa", "bookcase", "board", "clutter"
]
class2label = {cls: i for i, cls in enumerate(CLASSES)}

In [11]:
def parse_room(area_split, room_name, angle, align_angle=True, normalize=True):
    print(f"🔍 Parsing room: {room_name}")
    
    room_path = os.path.join(DATASET_ROOT, area_split, room_name)
    annotation_path = os.path.join(room_path, "Annotations")
    files = sorted(glob.glob(os.path.join(annotation_path, "*.txt")))

    if not files:
        print(f"⚠️ No annotation files in {annotation_path}")
        return None, None, None, None

    coords, colors, segment, instance = [], [], [], []

    for object_id, file in enumerate(files):
        class_name = os.path.basename(file).split("_")[0]
        label = class2label.get(class_name, class2label["clutter"])
        
        data = np.loadtxt(file)
        coords.append(data[:, :3])
        colors.append(data[:, 3:6] / 255.0)  # scale RGB to 0–1
        segment.append(np.full((data.shape[0], 1), label))
        instance.append(np.full((data.shape[0], 1), object_id))

    coords = np.vstack(coords)
    colors = np.vstack(colors)
    segment = np.vstack(segment)
    instance = np.vstack(instance)

    # Align coordinates using angle file
    if align_angle and angle is not None:
        angle_rad = (2 - angle / 180) * np.pi
        rot_cos, rot_sin = np.cos(angle_rad), np.sin(angle_rad)
        rot_matrix = np.array([
            [rot_cos, -rot_sin, 0],
            [rot_sin, rot_cos, 0],
            [0, 0, 1]
        ])
        room_center = (np.max(coords, axis=0) + np.min(coords, axis=0)) / 2
        coords = (coords - room_center) @ rot_matrix.T + room_center

    # ✅ Normalize coordinates
    if normalize:
        coords_mean = np.mean(coords, axis=0)
        coords_std = np.std(coords, axis=0)
        coords = (coords - coords_mean) / coords_std

    return coords, colors, segment, instance

In [ ]:
areas = ["Area_1", "Area_2", "Area_3", "Area_4", "Area_5", "Area_6"]

for area in areas:
    angle_file = os.path.join(DATASET_ROOT, area, f"{area}_alignmentAngle.txt")
    
    room_names, room_angles = [], []
    with open(angle_file, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 2:
                room_names.append(parts[0])
                room_angles.append(int(parts[1]))

    for room_name, angle in zip(room_names, room_angles):
        result = parse_room(area, room_name, angle, align_angle=True, normalize=True)
        if result[0] is None:
            continue
        
        coords, colors, segment, instance = result
        save_path = os.path.join(OUTPUT_ROOT, area, room_name)
        os.makedirs(save_path, exist_ok=True)

        np.save(os.path.join(save_path, "coord.npy"), coords.astype(np.float32))
        np.save(os.path.join(save_path, "color.npy"), (colors * 255).astype(np.uint8))
        np.save(os.path.join(save_path, "segment.npy"), segment.astype(np.int16))
        np.save(os.path.join(save_path, "instance.npy"), instance.astype(np.int16))

        print(f"✔ Saved: {area}/{room_name}")

🔍 Parsing room: conferenceRoom_1
✔ Saved: Area_1/conferenceRoom_1
🔍 Parsing room: conferenceRoom_2
✔ Saved: Area_1/conferenceRoom_2
🔍 Parsing room: copyRoom_1
✔ Saved: Area_1/copyRoom_1
🔍 Parsing room: hallway_1
✔ Saved: Area_1/hallway_1
🔍 Parsing room: hallway_2
✔ Saved: Area_1/hallway_2
🔍 Parsing room: hallway_3
✔ Saved: Area_1/hallway_3
🔍 Parsing room: hallway_4
✔ Saved: Area_1/hallway_4
🔍 Parsing room: hallway_5
✔ Saved: Area_1/hallway_5
🔍 Parsing room: hallway_6
✔ Saved: Area_1/hallway_6
🔍 Parsing room: hallway_7
✔ Saved: Area_1/hallway_7
🔍 Parsing room: hallway_8
✔ Saved: Area_1/hallway_8
🔍 Parsing room: office_1
✔ Saved: Area_1/office_1
🔍 Parsing room: office_2
✔ Saved: Area_1/office_2
🔍 Parsing room: office_3
✔ Saved: Area_1/office_3
🔍 Parsing room: office_4
✔ Saved: Area_1/office_4
🔍 Parsing room: office_5
✔ Saved: Area_1/office_5
🔍 Parsing room: office_6
✔ Saved: Area_1/office_6
🔍 Parsing room: office_7
✔ Saved: Area_1/office_7
🔍 Parsing room: office_8
✔ Saved: Area_1/office_

: 